# --- `Day 13`: Title ---

In [8]:
import aocd
import re
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [16]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [17]:
final_input = parse_input(aocd.get_data(day=13, year=2021))
print(final_input[:5])

['323,511', '1240,588', '1210,140', '641,365', '23,232']


In [18]:
test_input = parse_input('''\
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5
''')

print(test_input)

['6,10', '0,14', '9,10', '0,3', '10,4', '4,11', '6,0', '6,12', '4,1', '0,13', '10,12', '3,4', '3,0', '8,4', '1,10', '2,14', '8,10', '9,0', '', 'fold along y=7', 'fold along x=5']


### Helpers

In [82]:
def printBoard(board, marked = set()):
    spacing = 1
    for r in range(min(20, len(board))):
        print(''.join(str.rjust(str(board[r][c]), spacing)
                if (r,c) not in marked else str.rjust('*', spacing) 
                for c in range(min(40, len(board[0])))))
    print("")
    
def foldUp(board, fold):
    w,h = len(board[0]), len(board)
    result = [['.'] * w for _ in range(fold + 1)]
    for r in range(fold):
        for c in range(w):
            result[r][c] = board[r][c]
            
    pos = fold - 1
    for r in range(fold + 1, h):
        #print(pos,r)
        #print(result[pos], board[r])
        for j in range(w):
            if (board[r][j] == '#'):
                result[pos][j] = board[r][j]
        pos -= 1
        if pos < 0:
            break
            
    return result

def foldLeft(board, fold):
    w,h = len(board[0]), len(board)
    result = [['.'] * fold for _ in range(h)]
    
    for r in range(h):
        for c in range(fold):
            result[r][c] = board[r][c]
            
    pos = fold - 1
    for c in range(fold + 1, w):
        for j in range(h):
            if (board[j][c] == '#'):
                result[j][pos] = board[j][c]
        pos -= 1
        if pos < 0:
            break
    
    return result

## Solution 1

In [88]:
def solve_1(input):
    numbers = []
    folds = []
    readingBottom = False
    for line in input:
        if readingBottom:
            a,b = line.split('=')
            folds.append((a[-1], int(b)))
        elif line == "":
            readingBottom = True
        else:
            a,b = line.split(',')
            numbers.append((int(a), int(b)))
    #print(numbers)
    #print(folds)
    
    w = max(numbers)[0] + 1
    h = max(numbers, key=lambda x:x[1])[1] + 1
    board = [['.'] * w for _ in range(h)]
    
    print(w,h)
    for x,y in numbers:
        board[y][x] = '#'
        
    printBoard(board)
    
    for axis,f in folds:
        if axis == 'y':
            board = foldUp(board, f)
        elif axis == 'x':
            board = foldLeft(board, f)
        #break
    printBoard(board)
    
    count = 0
    for x in board:
        count += sum(a == '#' for a in x)
    return count

solve_1(test_input)

[(6, 10), (0, 14), (9, 10), (0, 3), (10, 4), (4, 11), (6, 0), (6, 12), (4, 1), (0, 13), (10, 12), (3, 4), (3, 0), (8, 4), (1, 10), (2, 14), (8, 10), (9, 0)]
[('y', 7), ('x', 5)]
11 15
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........

#####
#...#
#...#
#...#
#####
.....
.....
.....



16

In [89]:
f"Solution 1: {solve_1(final_input)}"

[(323, 511), (1240, 588), (1210, 140), (641, 365), (23, 232), (26, 448), (206, 791), (1208, 779), (1141, 210), (209, 796), (1012, 63), (1032, 700), (473, 327), (925, 120), (612, 669), (1148, 711), (333, 323), (405, 509), (577, 735), (315, 635), (1002, 50), (1158, 403), (388, 885), (746, 796), (709, 672), (929, 868), (224, 191), (679, 883), (939, 581), (442, 453), (380, 511), (136, 670), (455, 173), (1253, 423), (1019, 546), (1094, 764), (224, 555), (293, 834), (611, 278), (636, 844), (89, 799), (1052, 147), (1205, 603), (823, 287), (335, 767), (358, 438), (700, 275), (1116, 214), (919, 252), (485, 61), (462, 724), (303, 435), (1223, 621), (487, 738), (73, 227), (1057, 431), (667, 859), (965, 831), (1094, 549), (211, 143), (1141, 497), (67, 129), (631, 11), (961, 294), (448, 57), (25, 298), (641, 280), (216, 764), (1304, 318), (636, 443), (569, 679), (979, 679), (792, 709), (398, 739), (562, 709), (517, 746), (1143, 290), (1032, 271), (698, 469), (718, 585), (584, 739), (495, 287), (441

#..#.#..#.#..#...##..##...##....##.####.
#..#.#.#..#..#....#.#..#.#..#....#....#.
####.##...#..#....#.#....#..#....#...#..
#..#.#.#..#..#....#.#.##.####....#..#...
#..#.#.#..#..#.#..#.#..#.#..#.#..#.#....
#..#.#..#..##...##...###.#..#..##..####.
........................................



'Solution 1: 95'

## Solution 2

In [90]:
def solve_2(input):
    return ""
    
solve_1(test_input)

[(6, 10), (0, 14), (9, 10), (0, 3), (10, 4), (4, 11), (6, 0), (6, 12), (4, 1), (0, 13), (10, 12), (3, 4), (3, 0), (8, 4), (1, 10), (2, 14), (8, 10), (9, 0)]
[('y', 7), ('x', 5)]
11 15
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........

#####
#...#
#...#
#...#
#####
.....
.....
.....



16

In [91]:
f"Solution 2: {solve_1(final_input)}"

[(323, 511), (1240, 588), (1210, 140), (641, 365), (23, 232), (26, 448), (206, 791), (1208, 779), (1141, 210), (209, 796), (1012, 63), (1032, 700), (473, 327), (925, 120), (612, 669), (1148, 711), (333, 323), (405, 509), (577, 735), (315, 635), (1002, 50), (1158, 403), (388, 885), (746, 796), (709, 672), (929, 868), (224, 191), (679, 883), (939, 581), (442, 453), (380, 511), (136, 670), (455, 173), (1253, 423), (1019, 546), (1094, 764), (224, 555), (293, 834), (611, 278), (636, 844), (89, 799), (1052, 147), (1205, 603), (823, 287), (335, 767), (358, 438), (700, 275), (1116, 214), (919, 252), (485, 61), (462, 724), (303, 435), (1223, 621), (487, 738), (73, 227), (1057, 431), (667, 859), (965, 831), (1094, 549), (211, 143), (1141, 497), (67, 129), (631, 11), (961, 294), (448, 57), (25, 298), (641, 280), (216, 764), (1304, 318), (636, 443), (569, 679), (979, 679), (792, 709), (398, 739), (562, 709), (517, 746), (1143, 290), (1032, 271), (698, 469), (718, 585), (584, 739), (495, 287), (441

#..#.#..#.#..#...##..##...##....##.####.
#..#.#.#..#..#....#.#..#.#..#....#....#.
####.##...#..#....#.#....#..#....#...#..
#..#.#.#..#..#....#.#.##.####....#..#...
#..#.#.#..#..#.#..#.#..#.#..#.#..#.#....
#..#.#..#..##...##...###.#..#..##..####.
........................................



'Solution 2: 95'